Program untuk memisahkan beban listrik non hari libur nasional yang akan digunakan


Library :

In [1]:
import pandas as pd
import numpy as np
from National_Holiday import national_holiday
from Remove_Duplicate import Remove

Import Data :

In [2]:
path = r'kalbar1619.xlsx'
df01 = pd.read_excel (path)
df01.head()

,datetime,power
0,2016-01-01 01:00:00,189.997391
1,2016-01-01 02:00:00,182.481701
2,2016-01-01 03:00:00,174.183665
3,2016-01-01 04:00:00,170.306040
4,2016-01-01 05:00:00,171.152523


In [3]:
df01.tail()

,datetime,power
29803,2019-05-27 20:00:00,335.881022
29804,2019-05-27 21:00:00,333.506617
29805,2019-05-27 22:00:00,309.659184
29806,2019-05-27 23:00:00,282.947302
29807,2019-05-28 00:00:00,257.742326


Membuat Kolom Waktu :

In [4]:
df01['year'] = df01['datetime'].apply(lambda x: x.year)
df01['quarter'] = df01['datetime'].apply(lambda x: x.quarter)
df01['month'] = df01['datetime'].apply(lambda x: x.month)
df01['day'] = df01['datetime'].apply(lambda x: x.day)
df01["weekday"]=df01.apply(lambda row:row["datetime"].weekday(),axis=1)
df01["weekday"] = (df01["weekday"] < 5).astype(int)
df01["DayNumber"]=df01['datetime'].apply(lambda x: x.weekday())

In [5]:
df01.head(1)

,datetime,power,year,quarter,month,day,weekday,DayNumber
0,2016-01-01 01:00:00,189.997391,2016,1,1,1,1,4


DayNumber data awal adalah 4 (seharusnya 0), maka data awal akan dipotong

In [6]:
df01=df01.drop(df01.index[0:(((24*3)-1))])
df01.head(2)

,datetime,power,year,quarter,month,day,weekday,DayNumber
71,2016-01-04 00:00:00,164.514335,2016,1,1,4,1,0
72,2016-01-04 01:00:00,157.623233,2016,1,1,4,1,0


In [7]:
df01.tail(2)

,datetime,power,year,quarter,month,day,weekday,DayNumber
29806,2019-05-27 23:00:00,282.947302,2019,2,5,27,1,0
29807,2019-05-28 00:00:00,257.742326,2019,2,5,28,1,1


DayNumber data akhir adalah 1 (Seharusnya 6), maka data akhir juga dipotong

In [8]:
df01=df01.drop(df01.index[-24-1:])
df01.tail(2)

,datetime,power,year,quarter,month,day,weekday,DayNumber
29781,2019-05-26 22:00:00,287.074941,2019,2,5,26,0,6
29782,2019-05-26 23:00:00,262.454059,2019,2,5,26,0,6


In [9]:
df01 = df01.reset_index(drop=True)

Menandai Hari Libur Nasional 

In [10]:
data = df01.reset_index()
temp1 = len(df01)
temp2=[]
for i in range(temp1):
    temp3 = data.datetime.iloc[i]
    hari  = temp3.day
    bulan = temp3.month
    tahun = temp3.year
    weekday = data.weekday.iloc[i]
    temp4 = national_holiday(hari,bulan,tahun,weekday)
    temp2.append(temp4)

df01["National_Holiday"] = temp2

In [11]:
df01.head()

,datetime,power,year,quarter,month,day,weekday,DayNumber,National_Holiday
0,2016-01-04 00:00:00,164.514335,2016,1,1,4,1,0,0.0
1,2016-01-04 01:00:00,157.623233,2016,1,1,4,1,0,0.0
2,2016-01-04 02:00:00,153.295564,2016,1,1,4,1,0,0.0
3,2016-01-04 03:00:00,151.018003,2016,1,1,4,1,0,0.0
4,2016-01-04 04:00:00,144.734105,2016,1,1,4,1,0,0.0


In [12]:
df01["WeekNumber"] = 0

Membuat Nomer Minggu

In [13]:
a = 0
for i in range(len(df01)):
#     print(i)
    if ((df01.DayNumber.iloc[i] == 0) and (df01.datetime.iloc[i].hour == 0)) :
        a = a+1
        df01.WeekNumber.iloc[i] = a
    else :
        df01.WeekNumber.iloc[i] = a
df01.tail()

c:\users\microsoft user\appdata\local\programs\python\python37\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,datetime,power,year,quarter,month,day,weekday,DayNumber,National_Holiday,WeekNumber
29707,2019-05-26 19:00:00,297.121263,2019,2,5,26,0,6,0.0,177
29708,2019-05-26 20:00:00,312.122241,2019,2,5,26,0,6,0.0,177
29709,2019-05-26 21:00:00,308.669385,2019,2,5,26,0,6,0.0,177
29710,2019-05-26 22:00:00,287.074941,2019,2,5,26,0,6,0.0,177
29711,2019-05-26 23:00:00,262.454059,2019,2,5,26,0,6,0.0,177


Mencari Minggu yang terdapat hari libur nasional 

In [14]:
Delete_Week = []
for i in range(len(df01)):
    NH = df01.National_Holiday.iloc[i]
    WN = df01.WeekNumber.iloc[i]
    if NH == 1:
        Delete_Week.append(WN)
        
Delete_Week = Remove(Delete_Week)

Menghapus Minggu yang terdapat Hari Libur Nasional

In [15]:
for i in range(len(Delete_Week)):
    DW = Delete_Week[i]
    df01 = df01[df01.WeekNumber != DW]
df01 = df01.reset_index(drop=True)

In [16]:
df01.tail()

,datetime,power,year,quarter,month,day,weekday,DayNumber,National_Holiday,WeekNumber
22819,2019-05-26 19:00:00,297.121263,2019,2,5,26,0,6,0.0,177
22820,2019-05-26 20:00:00,312.122241,2019,2,5,26,0,6,0.0,177
22821,2019-05-26 21:00:00,308.669385,2019,2,5,26,0,6,0.0,177
22822,2019-05-26 22:00:00,287.074941,2019,2,5,26,0,6,0.0,177
22823,2019-05-26 23:00:00,262.454059,2019,2,5,26,0,6,0.0,177


In [17]:
a = 0
for i in range(len(df01)):
#     print(i)
    if ((df01.DayNumber.iloc[i] == 0) and (df01.datetime.iloc[i].hour == 0)) :
        a = a+1
        df01.WeekNumber.iloc[i] = a
    else :
        df01.WeekNumber.iloc[i] = a
df01 = df01.reset_index(drop=True)
df01.tail()

,datetime,power,year,quarter,month,day,weekday,DayNumber,National_Holiday,WeekNumber
22819,2019-05-26 19:00:00,297.121263,2019,2,5,26,0,6,0.0,136
22820,2019-05-26 20:00:00,312.122241,2019,2,5,26,0,6,0.0,136
22821,2019-05-26 21:00:00,308.669385,2019,2,5,26,0,6,0.0,136
22822,2019-05-26 22:00:00,287.074941,2019,2,5,26,0,6,0.0,136
22823,2019-05-26 23:00:00,262.454059,2019,2,5,26,0,6,0.0,136


Terdapat Data yang rusak pada Minggu ke 37, panjang data bukan 168, melainkan 144

In [18]:
Week37 = df01[df01.WeekNumber == 37]
len(Week37)

144

In [19]:
df01 = df01[df01.WeekNumber != 37]

In [20]:
a = 0
for i in range(len(df01)):
#     print(i)
    if ((df01.DayNumber.iloc[i] == 0) and (df01.datetime.iloc[i].hour == 0)) :
        a = a+1
        df01.WeekNumber.iloc[i] = a
    else :
        df01.WeekNumber.iloc[i] = a
df01 = df01.reset_index(drop=True)
df01.tail()

,datetime,power,year,quarter,month,day,weekday,DayNumber,National_Holiday,WeekNumber
22675,2019-05-26 19:00:00,297.121263,2019,2,5,26,0,6,0.0,135
22676,2019-05-26 20:00:00,312.122241,2019,2,5,26,0,6,0.0,135
22677,2019-05-26 21:00:00,308.669385,2019,2,5,26,0,6,0.0,135
22678,2019-05-26 22:00:00,287.074941,2019,2,5,26,0,6,0.0,135
22679,2019-05-26 23:00:00,262.454059,2019,2,5,26,0,6,0.0,135


In [21]:
df01 = df01.reset_index(drop=True)
df01.tail()

,datetime,power,year,quarter,month,day,weekday,DayNumber,National_Holiday,WeekNumber
22675,2019-05-26 19:00:00,297.121263,2019,2,5,26,0,6,0.0,135
22676,2019-05-26 20:00:00,312.122241,2019,2,5,26,0,6,0.0,135
22677,2019-05-26 21:00:00,308.669385,2019,2,5,26,0,6,0.0,135
22678,2019-05-26 22:00:00,287.074941,2019,2,5,26,0,6,0.0,135
22679,2019-05-26 23:00:00,262.454059,2019,2,5,26,0,6,0.0,135


Power Dataset :

In [22]:
df01.power

0        164.514335
1        157.623233
2        153.295564
3        151.018003
4        144.734105
            ...    
22675    297.121263
22676    312.122241
22677    308.669385
22678    287.074941
22679    262.454059
Name: power, Length: 22680, dtype: float64

In [23]:
# df01.power.to_excel("power_dataset.xlsx")  
# print("saved")